# DiffEdit

이미지 편집은 일반적으로 편집할 영역의 마스크를 제공해야 합니다. DiffEdit은 텍스트 쿼리를 기반으로 마스크를 자동으로 생성하므로 이미지 편집 소프트웨어 없이도 마스크를 더 쉽게 만들 수 있습니다. DiffEdit 알고리즘은 세 단계로 작동합니다.

1. 확산 모델은 일부 쿼리 텍스트와 참조 텍스트에 따라 이미지의 노이즈를 제거하여 이미지의 다른 영역에 대해 다른 노이즈 추정치를 생성합니다. 이 차이는 쿼리 텍스트와 일치하도록 변경해야 하는 이미지 영역을 식별하기 위한 마스크를 추론하는 데 사용됩니다.
2. 입력 이미지는 DDIM을 사용하여 잠재 공간으로 인코딩됩니다.
3. 잠재 변수는 텍스트 쿼리에 따라 확산 모델로 디코딩되며, 마스크를 가이드로 사용하여 마스크 외부의 픽셀은 입력 이미지와 동일하게 유지됩니다.

이 가이드에서는 DiffEdit을 사용하여 마스크를 수동으로 만들지 않고 이미지를 편집하는 방법을 보여줍니다.

시작하기 전에 다음 라이브러리가 설치되어 있는지 확인하십시오.

In [ ]:
# Colab에서 필요한 라이브러리를 설치하려면 주석 처리를 제거하십시오.
#!pip install -q diffusers transformers accelerate

[StableDiffusionDiffEditPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline)에는 이미지 마스크와 부분적으로 반전된 잠재 변수 집합이 필요합니다. 이미지 마스크는 [generate_mask()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline.generate_mask) 함수에서 생성되며 `source_prompt`와 `target_prompt`라는 두 가지 매개변수를 포함합니다. 이러한 매개변수는 이미지에서 편집할 내용을 결정합니다. 예를 들어, *과일* 한 그릇을 *배* 한 그릇으로 변경하려면 다음과 같이 합니다.

In [ ]:
source_prompt = "a bowl of fruits"
target_prompt = "a bowl of pears"

부분적으로 반전된 잠재 변수는 [invert()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline.invert) 함수에서 생성되며, 일반적으로 역 잠재 변수 샘플링 프로세스를 안내하는 데 도움이 되도록 이미지를 설명하는 `prompt` 또는 *캡션*을 포함하는 것이 좋습니다. 캡션은 종종 `source_prompt`가 될 수 있지만 다른 텍스트 설명으로 자유롭게 실험해 보세요!

파이프라인, 스케줄러, 역 스케줄러를 로드하고 메모리 사용량을 줄이기 위해 몇 가지 최적화를 활성화해 보겠습니다.

In [ ]:
import torch
from diffusers import DDIMScheduler, DDIMInverseScheduler, StableDiffusionDiffEditPipeline

pipeline = StableDiffusionDiffEditPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16,
    safety_checker=None,
    use_safetensors=True,
)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_model_cpu_offload()
pipeline.enable_vae_slicing()

편집할 이미지를 로드합니다.

In [ ]:
from diffusers.utils import load_image, make_image_grid

img_url = "https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"
raw_image = load_image(img_url).resize((768, 768))
raw_image

[generate_mask()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline.generate_mask) 함수를 사용하여 이미지 마스크를 생성합니다. 이미지에서 편집할 내용을 지정하려면 `source_prompt`와 `target_prompt`를 전달해야 합니다.

In [ ]:
from PIL import Image

source_prompt = "a bowl of fruits"
target_prompt = "a basket of pears"
mask_image = pipeline.generate_mask(
    image=raw_image,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
)
Image.fromarray((mask_image.squeeze()*255).astype("uint8"), "L").resize((768, 768))

다음으로, 반전된 잠재 변수를 만들고 이미지에 대한 설명을 전달합니다.

In [ ]:
inv_latents = pipeline.invert(prompt=source_prompt, image=raw_image).latents

마지막으로 이미지 마스크와 반전된 잠재 변수를 파이프라인에 전달합니다. 이제 `target_prompt`가 `prompt`가 되고 `source_prompt`가 `negative_prompt`로 사용됩니다.

In [ ]:
output_image = pipeline(
    prompt=target_prompt,
    mask_image=mask_image,
    image_latents=inv_latents,
    negative_prompt=source_prompt,
).images[0]
mask_image = Image.fromarray((mask_image.squeeze()*255).astype("uint8"), "L").resize((768, 768))
make_image_grid([raw_image, mask_image, output_image], rows=1, cols=3)

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">원본 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://github.com/Xiang-cd/DiffEdit-stable-diffusion/blob/main/assets/target.png?raw=true"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">편집된 이미지</figcaption>
  </div>
</div>

## 소스 및 타겟 임베딩 생성

소스 및 타겟 임베딩은 수동으로 만드는 대신 [Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) 모델을 사용하여 자동으로 생성할 수 있습니다.

🤗 Transformers 라이브러리에서 Flan-T5 모델과 토크나이저를 로드합니다.

In [ ]:
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

모델이 소스 및 타겟 프롬프트를 생성하도록 프롬프트할 초기 텍스트를 제공합니다.

In [ ]:
source_concept = "bowl"
target_concept = "basket"

source_text = f"Provide a caption for images containing a {source_concept}. "
"The captions should be in English and should be no longer than 150 characters."

target_text = f"Provide a caption for images containing a {target_concept}. "
"The captions should be in English and should be no longer than 150 characters."

다음으로 프롬프트를 생성하는 유틸리티 함수를 만듭니다.

In [ ]:
@torch.no_grad()
def generate_prompts(input_prompt):
    input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.to("cuda")

    outputs = model.generate(
        input_ids, temperature=0.8, num_return_sequences=16, do_sample=True, max_new_tokens=128, top_k=10
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

source_prompts = generate_prompts(source_text)
target_prompts = generate_prompts(target_text)
print(source_prompts)
print(target_prompts)

<Tip>

다양한 품질의 텍스트를 생성하는 전략에 대해 자세히 알아보려면 [생성 전략](https://huggingface.co/docs/transformers/main/en/generation_strategies) 가이드를 확인하세요.

</Tip>

[StableDiffusionDiffEditPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline)에서 사용하는 텍스트 인코더 모델을 로드하여 텍스트를 인코딩합니다. 텍스트 인코더를 사용하여 텍스트 임베딩을 계산합니다.

In [ ]:
import torch
from diffusers import StableDiffusionDiffEditPipeline

pipeline = StableDiffusionDiffEditPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, use_safetensors=True
)
pipeline.enable_model_cpu_offload()
pipeline.enable_vae_slicing()

@torch.no_grad()
def embed_prompts(sentences, tokenizer, text_encoder, device="cuda"):
    embeddings = []
    for sent in sentences:
        text_inputs = tokenizer(
            sent,
            padding="max_length",
            max_length=tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_input_ids = text_inputs.input_ids
        prompt_embeds = text_encoder(text_input_ids.to(device), attention_mask=None)[0]
        embeddings.append(prompt_embeds)
    return torch.concatenate(embeddings, dim=0).mean(dim=0).unsqueeze(0)

source_embeds = embed_prompts(source_prompts, pipeline.tokenizer, pipeline.text_encoder)
target_embeds = embed_prompts(target_prompts, pipeline.tokenizer, pipeline.text_encoder)

마지막으로 임베딩을 [generate_mask()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline.generate_mask) 및 [invert()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline.invert) 함수와 파이프라인에 전달하여 이미지를 생성합니다.

```diff
  from diffusers import DDIMInverseScheduler, DDIMScheduler
  from diffusers.utils import load_image, make_image_grid
  from PIL import Image

  pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
  pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)

  img_url = "https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"
  raw_image = load_image(img_url).resize((768, 768))

  mask_image = pipeline.generate_mask(
      image=raw_image,
-     source_prompt=source_prompt,
-     target_prompt=target_prompt,
+     source_prompt_embeds=source_embeds,
+     target_prompt_embeds=target_embeds,
  )

  inv_latents = pipeline.invert(
-     prompt=source_prompt,
+     prompt_embeds=source_embeds,
      image=raw_image,
  ).latents

  output_image = pipeline(
      mask_image=mask_image,
      image_latents=inv_latents,
-     prompt=target_prompt,
-     negative_prompt=source_prompt,
+     prompt_embeds=target_embeds,
+     negative_prompt_embeds=source_embeds,
  ).images[0]
  mask_image = Image.fromarray((mask_image.squeeze()*255).astype("uint8"), "L")
  make_image_grid([raw_image, mask_image, output_image], rows=1, cols=3)
```

## 반전을 위한 캡션 생성

`source_prompt`를 캡션으로 사용하여 부분적으로 반전된 잠재 변수를 생성하는 데 도움을 줄 수 있지만 [BLIP](https://huggingface.co/docs/transformers/model_doc/blip) 모델을 사용하여 캡션을 자동으로 생성할 수도 있습니다.

🤗 Transformers 라이브러리에서 BLIP 모델과 프로세서를 로드합니다.

In [ ]:
import torch
from transformers import BlipForConditionalGeneration, BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", torch_dtype=torch.float16, low_cpu_mem_usage=True)

입력 이미지에서 캡션을 생성하는 유틸리티 함수를 만듭니다.

In [ ]:
@torch.no_grad()
def generate_caption(images, caption_generator, caption_processor):
    text = "a photograph of"

    inputs = caption_processor(images, text, return_tensors="pt").to(device="cuda", dtype=caption_generator.dtype)
    caption_generator.to("cuda")
    outputs = caption_generator.generate(**inputs, max_new_tokens=128)

    # 캡션 생성기 오프로드
    caption_generator.to("cpu")

    caption = caption_processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return caption

`generate_caption` 함수를 사용하여 입력 이미지를 로드하고 캡션을 생성합니다.

In [ ]:
from diffusers.utils import load_image

img_url = "https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"
raw_image = load_image(img_url).resize((768, 768))
caption = generate_caption(raw_image, model, processor)

<div class="flex justify-center">
    <figure>
        <img class="rounded-xl" src="https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"/>
        <figcaption class="text-center">생성된 캡션: "테이블 위에 과일 한 그릇이 있는 사진"</figcaption>
    </figure>
</div>

이제 캡션을 [invert()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/diffedit#diffusers.StableDiffusionDiffEditPipeline.invert) 함수에 넣어 부분적으로 반전된 잠재 변수를 생성할 수 있습니다!